In [2]:
import psycopg2 as pg

In [9]:
conn = pg.connect(host='localhost', database='postgres', user='postgres')

cursor = conn.cursor()

In [56]:
def create_tables(cursor):
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS texts(
        id text PRIMARY KEY,
        author TEXT,
        year INTEGER,
        location TEXT,
        title TEXT
    )
    """)
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS reuses(
        source text REFERENCES texts(id),
        reuser text REFERENCES texts(id),
        count INTEGER,
        PRIMARY KEY (source, reuser)
    )
    """)

create_tables(cursor)

In [34]:
by_id = {}

with open('../data/text_with_details.csv') as f:
    f.readline()
    for line in f.readlines():
        id, title, year, place, author, *_ = line.split('|')

        if id in by_id:
            cur = by_id[id]
            cur['author'] = cur['author'] + '|||' + author.replace('\n', '')
        else:
            try:
                year_fix = ''.join([c for c in year if c.isdigit() or c == '.'])
                if year_fix[-1] == '.':
                    year_fix = year_fix[:-1]
                by_id[id] = {
                    'id': id,
                    'title': title,
                    'year': int(float(year_fix)),
                    'location': place,
                    'author': author.replace('\n', '')
                }
            except:
                pass

In [60]:
def add_texts(cursor):
    texts_arr = [val for k, val in by_id.items()]
    queries = [cursor.mogrify("INSERT INTO texts (id, author, year, location, title) VALUES (%s, %s, %s, %s, %s)", (t['id'], t['author'], t['year'], t['location'], t['title'])) for t in texts_arr]
    for query in queries:
        cursor.execute(query)
add_texts(cursor)    

UniqueViolation: duplicate key value violates unique constraint "texts_pkey"
DETAIL:  Key (id)=(N1001) already exists.


In [72]:
import time
def add_connections(cursor):
    with open('../data/cluster_connections_reduced.csv') as f:
        print(f.readline())
        cursor.copy_from(f, 'reuses', columns=('source', 'reuser', 'count'), sep=',')
        
        
add_connections(cursor)

source,reuser,count



In [55]:
cursor.execute('DROP TABLE reuses CASCADE;')

UndefinedTable: table "reuses" does not exist


In [10]:
cursor.execute("SELECT count(*) FROM reuses")
(cursor.fetchall())

[(285004,)]

In [80]:
cursor.execute

('T185733', 'T154179', 2132)